# Query by Donor Contribution Context #

## Overview ##

Query based on `ctx_donor_contrib` view

## Notebook Setup ##

### Configure database connect info/options ###

Note: database connect string can be specified on the initial `%sql` command:

```python
database_url = "postgresql+psycopg2://user@localhost/fecdb"
%sql $database_url

```

Or, connect string is taken from DATABASE_URL environment variable (if not specified for `%sql`):

```python
%sql

```

In [1]:
%load_ext sql
%config SqlMagic.autopandas=True
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
# connect string taken from DATABASE_URL environment variable
%sql

'Connected: crash@fecdb'

### Set styling ###

In [2]:
%%html
<style>
  tr, th, td {
    text-align: left !important;
  }
</style>

## Validate Context ##

In [3]:
%%sql
select count(*)
  from ctx_donor_contrib

 * postgresql+psycopg2://crash@localhost/fecdb
1 rows affected.


,count
0,4369


## Queries / Use Cases ##

### Contribution Summary by Election Cycle ###

In [4]:
%%sql
select dcx.elect_cycle,
       count(*) cycle_contribs,
       sum(dcx.transaction_amt) cycle_amount,
       round(avg(dcx.transaction_amt), 2) avg_amount,
       min(dcx.transaction_amt) min_amount,
       max(dcx.transaction_amt) max_amount
  from ctx_donor_contrib dcx
 group by 1
 order by 1

 * postgresql+psycopg2://crash@localhost/fecdb
10 rows affected.


,elect_cycle,cycle_contribs,cycle_amount,avg_amount,min_amount,max_amount
0,2002,5,3500.00,700.00,250.00,1000.00
1,2004,8,31000.00,3875.00,500.00,25000.00
2,2006,17,14200.00,835.29,200.00,2000.00
3,2008,22,13100.00,595.45,200.00,2300.00
4,2010,18,12250.00,680.56,250.00,2500.00
5,2012,19,21000.00,1105.26,250.00,10000.00
6,2014,322,635002.00,1972.06,-2600.00,100000.00
7,2016,1066,1493514.00,1401.05,-2700.00,73175.00
8,2018,2563,3892299.00,1518.65,-2744.00,102281.00
9,2020,329,547664.00,1664.63,-2800.00,50000.00


### Top Contributors across Election Cycles ###

In [5]:
%%sql
select d.id as donor_id,
       d.name as donor_name,
       count(*) contribs,
       sum(dcx.transaction_amt) total_amount,
       round(avg(dcx.transaction_amt), 2) avg_amount,
       max(dcx.transaction_amt) max_amount,
       array_agg(distinct dcx.elect_cycle) as elect_cycles,
       round(sum(dcx.transaction_amt) / count(distinct dcx.elect_cycle), 2) avg_cycle_amount
  from ctx_donor_contrib dcx
  join donor_indiv d on d.id = dcx.donor_indiv_id
 group by 1, 2
 order by 4 desc
 limit 50

 * postgresql+psycopg2://crash@localhost/fecdb
50 rows affected.


,donor_id,donor_name,contribs,total_amount,avg_amount,max_amount,elect_cycles,avg_cycle_amount
0,779629,"BEEUWKES, REINIER",433,888258.00,2051.40,100000.00,"[2006, 2008, 2010, 2012, 2014, 2016, 2018, 2020]",111032.25
1,6169689,"KARPLUS, BARBARA",340,878285.00,2583.19,73175.00,"[2016, 2018, 2020]",292761.67
2,10979072,"SHENKER, SCOTT",241,849800.00,3526.14,50000.00,"[2014, 2016, 2018, 2020]",212450.00
3,28429,"ABRAMSON, RONALD",295,471583.00,1598.59,33400.00,"[2012, 2014, 2016, 2018, 2020]",94316.60
4,1083690,"BLUE, ALLEN",81,426400.00,5264.20,33900.00,"[2016, 2018]",213200.00
5,7863787,"MCEVOY, NION",558,383899.00,687.99,50000.00,"[2002, 2004, 2006, 2008, 2010, 2012, 2014, 201...",38389.90
6,11902785,"TAYLOR, DALE",48,258050.00,5376.04,66100.00,"[2016, 2018, 2020]",86016.67
7,4336953,"GIRARDI, THOMAS V.",24,235100.00,9795.83,100000.00,"[2014, 2016, 2018, 2020]",58775.00
8,6433238,"KIRK, CLAY",98,211850.00,2161.73,20000.00,"[2014, 2016, 2018, 2020]",52962.50
9,1661575,"BYERS, BROOK",60,194900.00,3248.33,25000.00,"[2014, 2016, 2018, 2020]",48725.00


### Top Contributors within an Election Cycle ###

In [6]:
%%sql
select d.id as donor_id,
       d.name as donor_name,
       dcx.elect_cycle,
       count(*) cycle_contribs,
       sum(dcx.transaction_amt) cycle_total_amount,
       round(avg(dcx.transaction_amt), 2) cycle_avg_amount,
       max(dcx.transaction_amt) cycle_max_amount
  from ctx_donor_contrib dcx
  join donor_indiv d on d.id = dcx.donor_indiv_id
 group by 1, 2, 3
 order by 5 desc
 limit 50

 * postgresql+psycopg2://crash@localhost/fecdb
50 rows affected.


,donor_id,donor_name,elect_cycle,cycle_contribs,cycle_total_amount,cycle_avg_amount,cycle_max_amount
0,10979072,"SHENKER, SCOTT",2018,175,616700.00,3524.00,50000.00
1,6169689,"KARPLUS, BARBARA",2018,261,572685.00,2194.20,50000.00
2,779629,"BEEUWKES, REINIER",2018,206,493400.00,2395.15,100000.00
3,1083690,"BLUE, ALLEN",2016,62,288000.00,4645.16,33400.00
4,11902785,"TAYLOR, DALE",2018,38,227200.00,5978.95,66100.00
5,28429,"ABRAMSON, RONALD",2018,176,203524.00,1156.39,33400.00
6,6169689,"KARPLUS, BARBARA",2016,35,189400.00,5411.43,73175.00
7,779629,"BEEUWKES, REINIER",2016,141,185958.00,1318.85,8100.00
8,7863787,"MCEVOY, NION",2018,325,175294.00,539.37,44300.00
9,9188602,"PARK, TODD",2018,22,142214.00,6464.27,100000.00
